In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
def random_binning(N, d,return_string=False):
    """
    Function to distribute a sum total of N into d bins
    Returns a list
    """
    import numpy as np
    
    count=N
    sum_count=0
    lst=[]
    for i in range(d-1):
        x=np.random.randint(0,count+1)
        if return_string:
            lst.append(str(x))
        else:
            lst.append(x)
        count=count-x
        sum_count+=x
        
    if return_string:
        lst.append(str(N-sum_count))
    else:
        lst.append(N-sum_count)
    return lst

### Function to generate single term from a list of exponents. E.g. if we pass 1,2,0,3 it will generate: $x_1.x_2^2.x_4^3$

In [4]:
def gen_single_term(lst):
    """
    Function to generate single term from a list of exponents.
    """
    term=''
    for i in range(1,len(lst)+1):
        if lst[i-1]==0:
            pass
        elif lst[i-1]==1:
            term+='x'+str(i)+'.'
        else:
            term+='x'+str(i)+'^'+str(lst[i-1])+'.'
    
    return term[:-1]

In [5]:
gen_single_term([1,1,2,0,1])

'x1.x2.x3^2.x5'

In [6]:
def count_symbol(lst):
    syms = set(lst)
    syms_dict={}
    term_lst=[]
    for s in syms:
        syms_dict[s]=lst.count(s)
    
    for k,v in syms_dict.items():
        if v!=1:
            term_lst.append(str(v)+'.'+k)
        else:
            term_lst.append(k)
    
    return term_lst

In [7]:
def gen_multinomial(n_features=5,max_power=5,max_terms=10,fixed_terms=None,coefficients=True,
                   prob_negative_sign=0.3):
    """
    Generates multinomial expression.
    n_features: Number of independent variables
    max_power: Maximum exponent each terms can be raised to. A random power will be chosen up to this maximum.
    max_terms: Maximum number of terms. A random number of terms will be chosen up to this maximum.
    fixed_terms: Attempt will be made to generate only this many terms. Sometimes does not work.
    coefficients (boolean): Adds (or does not) random integer coefficients in front of the terms.
    prob_negative_sign: Probability of putting a negative term. 
                        Each term's sign wil be chosen randomly based on this probability
    """
    import numpy as np
    eqn=''
    eqn_terms=[]
    
    if fixed_terms!=None:
        n_terms=fixed_terms
    else:
        n_terms=np.random.randint(2,max_terms+1)
    
    for i in range(n_terms):
        power=np.random.randint(1,max_power+1)
        #power=max_power
        power_lst=random_binning(power,n_features)
        term=gen_single_term(power_lst)
        if coefficients:
            coeff=np.random.randint(1,11)
            if coeff!=1:
                coeff=str(coeff)
                term=coeff+'.'+term
        eqn_terms.append(term)
    
    eqn_terms=count_symbol(eqn_terms)
    
    for e in eqn_terms:
        eqn+=e
        sign=np.random.choice(['+','-'],p=[prob_negative_sign,1-prob_negative_sign])
        eqn= eqn+' '+sign+' '
      
    return eqn[:-3]

In [8]:
gen_multinomial(coefficients=True)

'6.x1^2.x2.x4 - 4.x1^2.x4^3 - 7.x1^2.x2^2 - 3.x1 - 7.x1^3.x3.x5 - 5.x1^5 - 6.x5'

In [9]:
def make_equation_system(n_samples=10,n_features=5,max_power=5,max_terms=10,
                               fixed_terms=None,coefficients=False,prob_negative_sign=0.3):
    """
    Generates multiple samples of multinomials for constructing a system of equations
    """
    regressions=[]
    for i in range(n_samples):
        regressions.append(gen_multinomial(n_features=n_features,max_power=max_power,
                                           max_terms=max_terms,fixed_terms=fixed_terms,
                                           coefficients=coefficients,prob_negative_sign=prob_negative_sign))
    
    return regressions

In [10]:
make_equation_system(n_samples=10,n_features=4,max_power=3,max_terms=8,fixed_terms=4,
                     coefficients=True,prob_negative_sign=0.5)

['10.x2 - x1 - 7.x1 - 6.x1',
 '2.x2.x4^2 - 2.x1 + 8.x1.x2.x4 - 10.x4',
 '8.x1^2 - 4.x1.x2 + 6.x1^3 + 4.x1',
 '2.x1^3 + 3.x1.x2^2 - 9.x1^2.x4 - 9.x1',
 '7.x1.x3 - x1^3 + x1^2.x3 + 3.x2^2.x3',
 '10.x2^2.x4 - 8.x2 + 5.x1.x3 + x2^2.x4',
 '2.x1^3 - x1.x4 - 5.x1^2 + 2.x3',
 '10.x1 + 4.x1.x2 - 9.x1^2.x4 + 3.x1^2.x4',
 'x2 - x1^3 + x2.x3^2 - 10.x1^2',
 '3.x1.x2.x4 - 3.x1^3 + 3.x1^2 - 3.x1']

### Evaluate a polynomial string

In [ ]:
from sympy import *

In [11]:
def symbolize(s):
    """
    Converts a a string (equation) to a SymPy symbol object
    """
    from sympy import sympify
    s1=s.replace('.','*')
    s2=s1.replace('^','**')
    s3=sympify(s2)
    
    return(s3)

In [22]:
def eval_multinomial(s,vals=None,symbolic_eval=False):
    """
    Evaluates polynomial at vals.
    vals can be simple list, dictionary, or tuple of values.
    vals can also contain symbols instead of real values provided those symbols have been declared before using SymPy
    """
    from sympy import Symbol,sympify,symbols
    
    sym_s=symbolize(s)
    sym_set=sym_s.atoms(Symbol)
    sym_lst=[]
    for s in sym_set:
        sym_lst.append(str(s))
    sym_lst.sort()
    if symbolic_eval==False and len(sym_set)!=len(vals):
        print("Length of the input values did not match number of variables and symbolic evaluation is not selected")
        return None
    else:
        if type(vals)==list:
            sub=list(zip(sym_lst,vals))
        elif type(vals)==dict:
            l=list(vals.keys())
            l.sort()
            lst=[]
            for i in l:
                lst.append(vals[i])
            sub=list(zip(sym_lst,lst))
        elif type(vals)==tuple:
            sub=list(zip(sym_lst,list(vals)))
        result=sym_s.subs(sub)
    
    return result

In [23]:
s1=gen_multinomial(fixed_terms=5,coefficients=True)

In [24]:
s1

'2.x4 - 6.x2^2.x3 - 10.x1 - 6.x3^2.x4^2 - 8.x1^3.x2.x5'

In [25]:
eval_multinomial(s1,{'x1':2,'x3':1,'x4':0,'x2':3,'x5':1},symbolic_eval=True)

-266

In [26]:
vals=(2,3,1)
eval_multinomial(s1,vals)

Length of the input values did not match number of variables and symbolic evaluation is not selected


In [27]:
from sympy import symbols
x,y=symbols('x y')
vals=[2,x,2,2,y]
eval_multinomial(s1,vals,symbolic_eval=True)

-12*x**2 - 64*x*y - 112

### Pretty printing and LaTeX format output

In [28]:
def pretty_multinomial(n_features=5,max_power=5,max_terms=10,fixed_terms=None,coefficients=False):
    from sympy import init_printing
    init_printing()
    s=gen_multinomial(n_features=n_features,max_power=max_power,max_terms=max_terms,
                      fixed_terms=fixed_terms,coefficients=coefficients)
    sym_s=symbolize(s)
    return(sym_s)

In [29]:
pretty_multinomial(coefficients=True,fixed_terms=4)

In [30]:
def latex_multinomial(n_features=5,max_power=5,max_terms=10,fixed_terms=None,coefficients=False):
    from sympy import latex
    s=gen_multinomial(n_features=n_features,max_power=max_power,max_terms=max_terms,
                      fixed_terms=fixed_terms,coefficients=coefficients)
    sym_s=symbolize(s)
    return(latex(sym_s))

In [31]:
latex_multinomial(coefficients=True,fixed_terms=4)

'5 x_{1}^{4} - 10 x_{1}^{2} x_{2} x_{5} + x_{1}^{2} x_{3} - 10 x_{2}^{3}'

In [32]:
lst=[]
for i in range(6):
    lst.append(pretty_multinomial(coefficients=True))
lst

In [33]:
pretty_multinomial(n_features=2,max_power=4,coefficients=True)